In [1]:
from ab_test_pipeline import Pipeline
import pandas as pd

res_alpha = pd.read_csv('res_alpha.csv', sep=';')

res_alpha.head()

,date,experimentId,experimentVariant,product,deviceCategory,userType,channel,goalID,sessions,users,conversions
0,23.09.2021,2ccd4f1b7d290c28600eb2a955f80242d96ad08d11c95a...,0,CC,mobile,Returning Visitor,channel1,goal1,1027,616,53
1,08.10.2021,2ccd4f1b7d290c28600eb2a955f80242d96ad08d11c95a...,0,CC,mobile,Returning Visitor,channel1,goal1,1160,806,56
2,26.09.2021,2ccd4f1b7d290c28600eb2a955f80242d96ad08d11c95a...,0,CC,mobile,Returning Visitor,channel1,goal1,1569,932,62
3,30.09.2021,2ccd4f1b7d290c28600eb2a955f80242d96ad08d11c95a...,0,CC,mobile,Returning Visitor,channel1,goal1,2330,1344,92
4,27.09.2021,2ccd4f1b7d290c28600eb2a955f80242d96ad08d11c95a...,0,CC,mobile,Returning Visitor,channel1,goal1,2087,1207,94


In [6]:
res_alpha['goalID'].unique()

array(['goal1', 'goal2'], dtype=object)

Вариант 1: вставлять цели в параметр groups с другими параметрами

In [ ]:
p = Pipeline(res_alpha)
res = p.pipeline('date', experiment_var_col="experimentVariant", 
                groups = ['deviceCategory', 'userType', 'goalID'],
                #metric_aggregations={"conversions":"sum", "sessions":"sum"},
               metrics_for_binary={"conversions": "sessions"})

In [8]:
print(res[0].to_string())

   deviceCategory           userType    goalID first second                   metric mean_lift   test_type   p_value
0        No group           No group  No group  df_0   df_1  conversions_sessions_CR  0.124456      z-test       0.0
1        No group           No group  No group  df_0   df_1  conversions_sessions_CR -0.054516  bayes_test       0.0
2        No group           No group  No group  df_0   df_2  conversions_sessions_CR  0.065156      z-test  0.001633
3        No group           No group  No group  df_0   df_2  conversions_sessions_CR  -0.02713  bayes_test  0.000824
4        No group           No group  No group  df_1   df_2  conversions_sessions_CR -0.052737      z-test   0.00201
5        No group           No group  No group  df_1   df_2  conversions_sessions_CR -0.026662  bayes_test  0.000997
6          mobile           No group  No group  df_0   df_1  conversions_sessions_CR  0.215272      z-test  0.026805
7          mobile           No group  No group  df_0   df_1  con

In [9]:
res[0]

,deviceCategory,userType,goalID,first,second,metric,mean_lift,test_type,p_value
0,No group,No group,No group,df_0,df_1,conversions_sessions_CR,0.124456,z-test,0.0
1,No group,No group,No group,df_0,df_1,conversions_sessions_CR,-0.054516,bayes_test,0.0
2,No group,No group,No group,df_0,df_2,conversions_sessions_CR,0.065156,z-test,0.001633
3,No group,No group,No group,df_0,df_2,conversions_sessions_CR,-0.02713,bayes_test,0.000824
4,No group,No group,No group,df_1,df_2,conversions_sessions_CR,-0.052737,z-test,0.00201
...,...,...,...,...,...,...,...,...,...
91,desktop,New Visitor,goal1,df_0,df_1,conversions_sessions_CR,0.027577,bayes_test,0.678877
92,tablet,Returning Visitor,goal1,df_0,df_1,conversions_sessions_CR,-0.713376,z-test,0.00001
93,tablet,Returning Visitor,goal1,df_0,df_1,conversions_sessions_CR,-0.569383,bayes_test,0.000005
94,tablet,New Visitor,goal1,df_0,df_1,conversions_sessions_CR,0.226049,z-test,0.308565


Вариант 2: цикл по уникальным целям вне pipeline

In [3]:
goals = res_alpha['goalID'].unique()
goals

array(['goal1', 'goal2'], dtype=object)

In [17]:
for i,x in enumerate(goals):
    if i==0:
        print(i)
    print(i)

0
0
1


In [4]:

for i,x in enumerate(goals):
    if i==0:
        p = Pipeline(res_alpha[res_alpha['goalID']==x])
        res = p.pipeline('date', experiment_var_col="experimentVariant", 
                        groups = ['deviceCategory', 'userType'],
                        #metric_aggregations={"conversions":"sum", "sessions":"sum"},
                    metrics_for_binary={"conversions": "sessions"})[0]
        res['goalsID'] = x
    else:
        p = Pipeline(res_alpha[res_alpha['goalID']==x])
        temp = p.pipeline('date', experiment_var_col="experimentVariant", 
                        groups = ['deviceCategory', 'userType'],
                        #metric_aggregations={"conversions":"sum", "sessions":"sum"},
                    metrics_for_binary={"conversions": "sessions"})[0]
        temp['goalsID'] = x
        res = res.append(temp)

res

c:\Users\user1676\Работа\RwAB\RwAB\ab_test_pipeline.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1[cur_metric] = df_1[succ] / df_1[trial]
c:\Users\user1676\Работа\RwAB\RwAB\ab_test_pipeline.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2[cur_metric] = df_2[succ] / df_2[trial]
c:\Users\user1676\Работа\RwAB\RwAB\ab_test_pipeline.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

,deviceCategory,userType,first,second,metric,mean_lift,test_type,p_value,goalsID
0,No group,No group,df_0,df_1,conversions_sessions_CR,0.124456,z-test,0.0,goal1
1,No group,No group,df_0,df_1,conversions_sessions_CR,-0.054516,bayes_test,0.0,goal1
2,mobile,No group,df_0,df_1,conversions_sessions_CR,0.215272,z-test,0.026805,goal1
3,mobile,No group,df_0,df_1,conversions_sessions_CR,-0.046016,bayes_test,0.013401,goal1
4,desktop,No group,df_0,df_1,conversions_sessions_CR,0.161946,z-test,0.0,goal1
5,desktop,No group,df_0,df_1,conversions_sessions_CR,-0.079734,bayes_test,0.0,goal1
6,tablet,No group,df_0,df_1,conversions_sessions_CR,-0.677782,z-test,0.000082,goal1
7,tablet,No group,df_0,df_1,conversions_sessions_CR,-0.463815,bayes_test,0.000039,goal1
8,No group,Returning Visitor,df_0,df_1,conversions_sessions_CR,0.127523,z-test,0.0,goal1
9,No group,Returning Visitor,df_0,df_1,conversions_sessions_CR,-0.059578,bayes_test,0.0,goal1


In [27]:
p = Pipeline(res_alpha[res_alpha['goalID']==goals[1]])
temp = p.pipeline('date', experiment_var_col="experimentVariant", 
                groups = ['deviceCategory', 'userType'],
                #metric_aggregations={"conversions":"sum", "sessions":"sum"},
            metrics_for_binary={"conversions": "sessions"})[0]

In [28]:
res_alpha[res_alpha['goalID']==goals[1]]

,date,experimentId,experimentVariant,product,deviceCategory,userType,channel,goalID,sessions,users,conversions
2314,23.09.2021,2ccd4f1b7d290c28600eb2a955f80242d96ad08d11c95a...,2,CC,mobile,Returning Visitor,channel1,goal2,1027,616,53
2315,08.10.2021,2ccd4f1b7d290c28600eb2a955f80242d96ad08d11c95a...,2,CC,mobile,Returning Visitor,channel1,goal2,1160,806,56
2316,26.09.2021,2ccd4f1b7d290c28600eb2a955f80242d96ad08d11c95a...,2,CC,mobile,Returning Visitor,channel1,goal2,1569,932,62
2317,30.09.2021,2ccd4f1b7d290c28600eb2a955f80242d96ad08d11c95a...,2,CC,mobile,Returning Visitor,channel1,goal2,2330,1344,92
2318,27.09.2021,2ccd4f1b7d290c28600eb2a955f80242d96ad08d11c95a...,2,CC,mobile,Returning Visitor,channel1,goal2,2087,1207,94
...,...,...,...,...,...,...,...,...,...,...,...
4623,21.10.2021,397c4f579c2c394bfd4d2b53d0d8266b4d3a0637d20270...,2,CC,desktop,Returning Visitor,channel2,goal2,599,289,31
4624,22.10.2021,397c4f579c2c394bfd4d2b53d0d8266b4d3a0637d20270...,2,CC,desktop,Returning Visitor,channel2,goal2,495,256,31
4625,04.11.2021,397c4f579c2c394bfd4d2b53d0d8266b4d3a0637d20270...,2,CC,desktop,Returning Visitor,channel2,goal2,527,249,31
4626,13.11.2021,397c4f579c2c394bfd4d2b53d0d8266b4d3a0637d20270...,2,CC,desktop,Returning Visitor,channel2,goal2,411,200,31
